## 19대 국회 회의록 데이터

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random, unicodedata
from Mylib import *
import os

Start loading Mylib...
Mylib imported!


In [98]:
files = [file_name for file_name in os.listdir('./data') if '.csv' in file_name]
국회_19대 = pd.read_csv(f'./data/{files[0]}', encoding = 'utf-8')
for file in files[1:]:
    new = pd.read_csv(f'./data/{file}', encoding = 'utf-8')
    국회_19대 = 국회_19대.append(new, ignore_index = True)

In [102]:
국회_19대.발언내용 = [str(e) for e in 국회_19대.발언내용]
국회_19대.groupby(['위원회']).size()

위원회
교육과학기술위원회        16480
교육문화체육관광위원회      52770
국방위원회            60860
국토교통위원회          32119
국토해양위원회          11715
국회운영위원회          28691
기획재정위원회          39169
농림수산식품위원회         7976
농림축산식품해양수산위원회    41617
문화체육관광방송통신위원회    17129
미래창조과학방송통신위원회    42680
법제사법위원회          89074
보건복지위원회          42985
산업통상자원위원회        35459
안전행정위원회          43773
여성가족위원회          12873
외교통상통일위원회        14791
외교통일위원회          26734
정무위원회            45740
정보위원회             3522
지식경제위원회          12613
행정안전위원회          16303
환경노동위원회          42881
dtype: int64

In [111]:
국회_19대 = 국회_19대.loc[[i for i, j in 국회_19대.발언내용.items() if j != 'nan']]

#### Get NNP + NNGs for LDA and Network

In [123]:
data_for_noun = 국회_19대.assign(token = [get_noun(doc)
                                       for doc in tqdm(국회_19대.발언내용.values)])
data_for_noun = data_for_noun.loc[[i for i, tkn in data_for_noun.token.items() 
                                   if len(tkn) > 0]]
data_for_noun.to_pickle("./result/국회_19대_토큰.pkl") # for LDA

# data_for_noun = data_for_noun.drop('token', 1)
# data_for_noun = data_for_noun.assign(token = [[get_noun(doc) for doc in re.split('[?.!]', corp)] 
#                                               for corp in tqdm(data_for_noun.CONTENT.values)])
# data_for_noun.to_pickle('./result/국회_19대_토큰_NW.pkl') # for Network

100%|██████████| 737951/737951 [51:31<00:00, 238.68it/s]  


#### 1차 불용어 필터링
* 형태소를 추출한 뒤 빈도 기준 상위 1,000개의 토큰에서 정보량이 적은 토큰을 제거

In [3]:
data_for_noun = pd.read_pickle("./result/국회_19대_토큰.pkl")

In [4]:
cnt = Counter(my_list([doc for doc in data_for_noun.token]).sum())
frqtab = sorted(cnt.items(), key = lambda x: x[1], reverse = True)[:1000]

In [5]:
exclude = '말씀 생각 부분 다음 때 얘기 내용 경우 정도 이상 말 감사 항 수고 의견 앞 부 안 노력 이번 전 점 사항 이야기 존경 지금 예 위 그때 날 발 조 도 후 장 끝 원 오늘 방 간 뜻 뒤 실 시 곳 미 면 배 급 속 인 수 처 반 밖 직 관 답 사 청'.split() 
frqtab = [k for k, f in frqtab if re.search('|'.join(exclude), k)]
data_for_noun.token = [[tkn for tkn in doc if tkn not in exclude]
                       for doc in tqdm(data_for_noun.token)]
data_for_noun = data_for_noun.loc[[i for i, tkn in data_for_noun.token.items() 
                                   if len(tkn) > 0]]

100%|██████████| 605284/605284 [00:17<00:00, 33659.37it/s]


In [6]:
data_for_noun

,회의록ID,대수,위원회구분,위원회,회차,회의일자,회의장소,발언ID,발언순서,안건,발언자,발언내용,token
0,41725.0,19.0,상임위원회,국회운영위원회,제309회(2012.07.05-2012.08.03) - 제1차,2012년7월10일(화),문화체육관광방송통신위원회회의실,814715.0,1.0,FRONT,위원장 이한구,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제309회 국회(임시회) 제1...,"[의석, 정돈, 성원, 국회, 임시회, 국회, 운영, 위원회, 개의, 입, 법조, ..."
1,41725.0,19.0,상임위원회,국회운영위원회,제309회(2012.07.05-2012.08.03) - 제1차,2012년7월10일(화),문화체육관광방송통신위원회회의실,814716.0,2.0,FRONT,김기현 위원,반갑습니다. 앉아서 인사드리겠습니다. 새누리당 김기현 위원입니다. 앞으로 우리...,"[인사, 누리, 당, 김기현, 위원, 위원, 위원회, 국회, 모범, 위원회, 운영,..."
2,41725.0,19.0,상임위원회,국회운영위원회,제309회(2012.07.05-2012.08.03) - 제1차,2012년7월10일(화),문화체육관광방송통신위원회회의실,814717.0,3.0,FRONT,위원장 이한구,인사를 하면 박수부터 치고 합시다. (일동 박수),"[인사, 박수, 일동, 박수]"
3,41725.0,19.0,상임위원회,국회운영위원회,제309회(2012.07.05-2012.08.03) - 제1차,2012년7월10일(화),문화체육관광방송통신위원회회의실,814718.0,4.0,FRONT,김기선 위원,새누리당의 원주 출신 김기선 위원입니다. 앉아서 인사드리겠습니다. 특히 선배 위...,"[누리, 당, 원주, 출신, 김기선, 위원, 인사, 선배, 위원, 국회, 국민, 기..."
4,41725.0,19.0,상임위원회,국회운영위원회,제309회(2012.07.05-2012.08.03) - 제1차,2012년7월10일(화),문화체육관광방송통신위원회회의실,814719.0,5.0,FRONT,김도읍 위원,반갑습니다. 부산 북구․강서구을 출신 새누리당 김도읍 위원입니다. 여러 선배 위...,"[부산, 북구, 강서구, 출신, 새, 김도읍, 위원, 선배, 위원, 동료, 위원, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
976105,42575.0,19.0,상임위원회,교육과학기술위원회,제314회(2013.03.08-2013.03.22) - 제1차,2013년3월19일(화),교육과학기술위원회회의실,996236.0,16.0,1. 국가교육과학기술자문회의법 일부개정법률안(이한구 의원 대표발의)(이한구ㆍ황우여ㆍ...,金世淵 委員,제가 한 가지 당부 말씀……,[당부]
976107,42575.0,19.0,상임위원회,교육과학기술위원회,제314회(2013.03.08-2013.03.22) - 제1차,2013년3월19일(화),교육과학기술위원회회의실,996238.0,18.0,1. 국가교육과학기술자문회의법 일부개정법률안(이한구 의원 대표발의)(이한구ㆍ황우여ㆍ...,金世淵 委員,"우리 교과위에서 그동안 정부조직법 개편 관련해서, 특히 다른 상임위보다는 많은 법률...","[교과, 그동안, 정부, 조직법, 개편, 관련, 상임위, 법률, 상당, 진통, 기간..."
976109,42575.0,19.0,상임위원회,교육과학기술위원회,제314회(2013.03.08-2013.03.22) - 제1차,2013년3월19일(화),교육과학기술위원회회의실,996240.0,20.0,1. 국가교육과학기술자문회의법 일부개정법률안(이한구 의원 대표발의)(이한구ㆍ황우여ㆍ...,위원장직무대리 유기홍,"예, 강은희 위원님.","[강은희, 위원]"
976110,42575.0,19.0,상임위원회,교육과학기술위원회,제314회(2013.03.08-2013.03.22) - 제1차,2013년3월19일(화),교육과학기술위원회회의실,996241.0,21.0,1. 국가교육과학기술자문회의법 일부개정법률안(이한구 의원 대표발의)(이한구ㆍ황우여ㆍ...,강은희 위원,교육부장관님께 부탁 말씀을 드리겠습니다. 오늘 이 자리에 말씀드릴 일은 전혀 아니...,"[교육부, 장관, 부탁, 자리, 일, 대학, 평가, 교육, 역량, 강화, 사업, 링..."


#### BOW 모형을 이용해 문서와 어휘집을 pickle로 저장

In [39]:
국회_19대_bow = BowModel(data_for_noun.token)
국회_19대_bow.get_bow(.0, 1.0)

pd.to_pickle(data_for_noun, './result/국회_19대_meta.pickle')
pd.to_pickle(국회_19대_bow.bow_corpus, './result/국회_19대_BOW.pickle')
pd.to_pickle([v for v in 국회_19대_bow.dictionary.values()], './result/국회_19대_dict.pickle')

#### 문서의 길이
말뭉치 결합 $\rightarrow$ 명사 토큰 N = 100 기준으로 재결합 $\rightarrow$ D > 2000? $\rightarrow$ Fit!\
"... it is theoretically $impossible$ to guarantee identification of topics from a small number of documents, no matter how long."\
"... poor performance of the LDA is expected when documents are too short, ..."
* 참고문헌: Jian Tang et al. (2014) Understanding the limiting factors of topic modeling via posterior contraction analysis. $\textit{In Proceedings of the 31st International Conference on International Conference on Machine Learning}$ - Volume 32 (ICML’14). JMLR.org, I–190–I–198.

In [131]:
def nrmlzcorp(corpus, thrsh = 100):
    merged_tokens = my_list(corpus).sum()
    nrmcorp, i, stop = [[]], 0, False
    while not stop:
        while len(nrmcorp[-1]) < thrsh and i < len(merged_tokens):
            nrmcorp[-1].append(merged_tokens[i])
            i += 1
            if i == len(merged_tokens):
                stop = True
        nrmcorp.append([])
    return(nrmcorp[:-1])

In [ ]:
nrmcorp = nrmlzcorp(국회_19대.token)